In [97]:
from parrot import Parrot
import pandas as pd
from tqdm import tqdm

In [68]:
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=True)

In [76]:
def paraphrase_aug(text):
    paraphrases = parrot.augment(text,  max_length = 250, max_return_phrases = 5, do_diverse=False)
    return paraphrases

In [5]:
data = pd.read_csv("/workspaces/MBTI-Personality-Test/Data/mbti_train.csv")

In [7]:
cnt = data.groupby("label").count()
avg = len(data) // 16
print(cnt, avg)

        text
label       
0       7639
1       6279
2      55830
3      40742
4      12630
5       9950
6      69372
7      48976
8       3391
9       1731
10     25672
11     26415
12      1507
13      1631
14      7311
15      8752 20489


In [10]:
aug = cnt.apply(lambda x: (avg - x)).clip(0)
aug

,text
label,
0,12850
1,14210
2,0
3,0
4,7859
5,10539
6,0
7,0
8,17098


In [23]:
data0 = data[data["label"] == 0].reset_index(drop=True)

In [24]:
data0.head()

,label,text
0,0,Seeing as how you can't go for even one post w...
1,0,"Been reading up on the Beatles some more, litt..."
2,0,Remember to cheat and use sun glasses.
3,0,Hello and welcome to the forum.
4,0,"Yes, I even do this in supposedly factual prog..."


In [95]:
def augment_data(data, label):
    aug = {label: []}
    for i in tqdm(range(len(data))):
        paras = paraphrase_aug(data[i])
        if paras:
            for para in paras:
                if para[1] != 0:
                    aug[label].append(para[0])
    return pd.DataFrame(aug)

In [91]:
d = data[data['label'] == 12].reset_index(drop=True)

In [93]:
d.shape

(1507, 2)

In [98]:
t = augment_data(d['text'], 12)

  2%|▏         | 28/1507 [02:59<2:38:27,  6.43s/it]


KeyboardInterrupt: 